<a href="https://colab.research.google.com/github/ramkumardeepak774/similarity_Search_using_Faiss/blob/main/search_query_using_faiss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import requests
from io import StringIO
import pandas as pd

In [20]:
!pip install faiss-cpu



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 38.4 MB/s eta 0:00:00


In [21]:
import faiss


In [22]:
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')

text = res.text
text[:100]

'pair_ID\tsentence_A\tsentence_B\trelatedness_score\tentailment_judgment\n1\tA group of kids is playing in '

In [23]:
# We need this in a dataframe, which we build from the text string like so:
data = pd.read_csv(StringIO(text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [24]:
sentences = data['sentence_A'].tolist()
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)
len(set(sentences))

4802

In [25]:
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [55]:
# so let's pull in a few more similar datasets.
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

In [56]:
# each of these dataset have the same structure, so we loop through each creating our sentences data
for url in urls:
    res = requests.get(url)
    # extract to dataframe
    data = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
    # add to columns 1 and 2 to sentences list
    sentences.extend(data[1].tolist())
    sentences.extend(data[2].tolist())

<ipython-input-56-55e57fcc44d7>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
Skipping line 191: expected 3 fields, saw 4
Skipping line 206: expected 3 fields, saw 4
Skipping line 295: expected 3 fields, saw 4
Skipping line 695: expected 3 fields, saw 4
Skipping line 699: expected 3 fields, saw 4

<ipython-input-56-55e57fcc44d7>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
Skipping line 104: expected 3 fields, saw 4
Skipping line 181: expected 3 fields, saw 4
Skipping line 317: expected 3 fields, saw 4
Skipping line 412: expected 3 fields, saw 5
Skipping line 508: expected 3 fields, saw 4

<ipython-input-56-55e57fcc44d7>:5: F

In [57]:
len(set(sentences))

14505

In [58]:
# remove duplicates and NaN
sentences = [word for word in list(set(sentences)) if type(word) is str]

In [59]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [60]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

In [61]:
sentence_embeddings = model.encode(sentences)


In [62]:
dim = len(sentence_embeddings[0])

In [63]:
index = faiss.IndexFlatL2(dim)


In [64]:
index.is_trained

True

In [65]:
index.add(sentence_embeddings)

In [66]:
index.ntotal

14504

In [67]:
# Then search given a query xq and number of nearest neigbors to return k
k = 4
xq = model.encode(["Someone sprints with a football"])


In [68]:
%%time
D, I = index.search(xq, k)  # search
print(I)  # k-nearest neigbors of the query vector 

[[  706 11562  5585  8177]]
CPU times: user 5 ms, sys: 0 ns, total: 5 ms
Wall time: 5.09 ms


In [69]:
[f'{i}: {sentences[i]}' for i in I[0]]


['706: A group of people playing football is running in the field',
 '11562: A person playing football is running past an official carrying a football',
 '5585: A group of football players is running in the field',
 '8177: A football player is running past an official carrying a football']

In [70]:
#another query 
k = 4
xq1 = model.encode(["i don't like football"])

In [71]:
%%time
D, I = index.search(xq1, k)  # search
print(I)  # k-nearest neigbors of the query vector 

[[12663 13863 10054  1060]]
CPU times: user 6.27 ms, sys: 38 µs, total: 6.3 ms
Wall time: 29.4 ms


Result using Faiss similarity search

In [72]:
[f'{i}: {sentences[i]}' for i in I[0]]


['12663: The crowd is not watching a football game',
 '13863: The crowd is watching a football game',
 '10054: The crowd is watching the football at the game',
 '1060: Young boys play touch football.']

Calculating the euclid distance of query to each vector of senctence is more expensive thats why we use 

**Partitioning The Index **

Faiss allows us to add multiple steps that can optimize our search using many different methods. A popular approach is to partition the index into Voronoi cells.



In [81]:
# We use nlist to specify how many partitions (Voronoi cells) we’d like our index to have
nlist = 50  # how many cells
quantizer = faiss.IndexFlatL2(dim)
index = faiss.IndexIVFFlat(quantizer, dim, nlist)

In [82]:
index.train(sentence_embeddings)
index.is_trained  # check if index is now trained

True

In [83]:
index.add(sentence_embeddings)
index.ntotal  # number of embeddings indexed

14504

In [84]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[  706 11562  5585  8177]]
CPU times: user 2.36 ms, sys: 12 µs, total: 2.37 ms
Wall time: 2.39 ms


In [85]:
%%time
D, I = index.search(xq1, k)  # search
print(I)

[[12663 13863 10054  1060]]
CPU times: user 653 µs, sys: 2.03 ms, total: 2.68 ms
Wall time: 5.66 ms


Result using cosine *similarity* 

In [40]:
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [42]:
from sentence_transformers import util

In [86]:
# Calculate the cosine similarity between the query and each sentence in the corpus
%%time
top_result_xq = util.semantic_search(xq, sentence_embeddings, top_k = k )

CPU times: user 13.7 ms, sys: 0 ns, total: 13.7 ms
Wall time: 17.3 ms


In [87]:
top_result_xq

[[{'corpus_id': 706, 'score': 0.6178056001663208},
  {'corpus_id': 11562, 'score': 0.6176843643188477},
  {'corpus_id': 5585, 'score': 0.603889524936676},
  {'corpus_id': 8177, 'score': 0.5928354859352112}]]

In [76]:
for index in top_result_xq[0]:
  print(sentences[index['corpus_id']])

A group of people playing football is running in the field
A person playing football is running past an official carrying a football
A group of football players is running in the field
A football player is running past an official carrying a football


In [88]:
#cosine query for xq1
%%time
top_result_xq1 = util.semantic_search(xq1, sentence_embeddings, top_k = k )

CPU times: user 12.3 ms, sys: 0 ns, total: 12.3 ms
Wall time: 17.3 ms


In [78]:
top_result_xq1

[[{'corpus_id': 12663, 'score': 0.5211633443832397},
  {'corpus_id': 13863, 'score': 0.513023316860199},
  {'corpus_id': 10054, 'score': 0.5006490349769592},
  {'corpus_id': 1060, 'score': 0.4843605160713196}]]

In [80]:
for index in top_result_xq1[0]:
  print(sentences[index['corpus_id']])

The crowd is not watching a football game
The crowd is watching a football game
The crowd is watching the football at the game
Young boys play touch football.
